# Model Identification for OnInvoice Discount Prediction

    1) All the parameters used in prediction of Total Discount can be used in predicting OnInvoice Discount

    2) Another very important parameter that we think is important for predicting OnInvoice Discounts is the term period   of   Invoice
    
To understand this let's understand how invoice Discounts work.

Businesses want immediate cash flow on their orders and hence can't wait for their customers to pay them money. So they go to a financing company & sell invoices for which they get a percentage of total Invoice value as initial payment . Financing Company now witholds the responsibility of getting the money from the customer. After customer pays the money to the financing company , the financing company returns the remaining amount after deducting service charges.

Typically financing company agrees to give 30 days , 60 days , 90 days ..... time as options for the customer to pay them back . 
Higher time the customer takes to payback , higher is the service charge deducted by the financing company. So if we customers agree to pay in a short time period , our service charges will be less and hence we can give such customers higher discounts.


However we don't have data for the time period of Invoice but we think having that data will help us predict OnInvoice Discounts much more accurately

In [2]:
#### IMPORTS

import pandas as pd
import numpy as np
from types import FunctionType
from sklearn.model_selection import train_test_split
import math
import matplotlib.pyplot as plt
# Importing sklearn methods
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn import tree

In [3]:
# data2.xlsx is the data obtained after running the feature engineering code
# data3.csv : converted data2.xlsx to data3.csv because of easiness of use of csv files in webapps
path = r"C:\Users\NISARG\Desktop\mech\Finance\Maverick\CODE"   #change the path to your local path
df = pd.read_csv(path + "\data3.csv")
df.head()


Unnamed: 0  Unnamed: 0.1  Ship-to ID  Volume_2019  Volume_2018 sdfc_Tier  \
0           0             0    29000310         0.48        0.557    Tier 0   
1           1             1    29000419         0.45        0.540    Tier 1   
2           2             2    29000430       270.97      225.720    Tier 1   
3           3             3    29000430       270.97      225.720    Tier 1   
4           4             4    29000430       270.97      225.720    Tier 1   

    poc_image            segment     sub_segment  \
0  Mainstream  Entertainment Led          Events   
1  Mainstream     Not applicable  Not applicable   
2  Mainstream          Drink Led     Party Place   
3  Mainstream          Drink Led     Party Place   
4  Mainstream          Drink Led     Party Place   

                                    Product Set  ...  Expected_GTO  \
0        RETURNABLE_BOTTLE_JUPILER_JUPILER PILS  ...    116.362876   
1  RETURNABLE_BOTTLE_PIEDBOEUF_PIEDBOEUF TRIPLE  ...     88.943478   
2                  OW_BULK_JUPILER_JUPILER PILS  ...  71342.136430   
3        RETURNABLE_BOTTLE_JUPILER_JUPILER PILS  ...   6955.593627   
4           RETURNABLE_KEG_JUPILER_JUPILER PILS  ...   3536.747237   

  Expected_product_volume loyalty_index min_order_size_for_discount  \
0                0.395568             0                    0.000059   
1                0.352174             0                    0.002273   
2              276.519909             1                    0.000768   
3               23.645077             1                    0.000059   
4               13.908869             0                    0.000064   

   inventory_lingering_factor  profit_Product  profitability_indicator  \
0                    0.030049      370.415207                 0.474992   
1                    0.057372      207.180476                 0.265672   
2                   60.857752    77983.465190               100.000000   
3                    0.507591      370.415207                 0.474992   
4                    5.144873     3160.708348                 4.053049   

    discount_std   upper_limit Disc_Percent  
0     840.354946     30.119944     0.000000  
1     421.813020     21.621963     0.000000  
2  188693.263300  39604.153890     0.235763  
3     840.354946   1800.544853     0.267487  
4   11271.537870    948.335453     0.235409  

[5 rows x 36 columns]

In [3]:
count = 0
index_test = list()
index_train = list()
for i in range(len(df['upper_limit'])):
    if(df['Discount_Total'][i]>df['upper_limit'][i]):
        count+=1
        index_test.append(i)
    else:
        index_train.append(i)
print(count/len(df['upper_limit']))

0.3232645073885446


In [4]:
df_test = df.iloc[index_test]
df_test = df_test.reset_index()
df_train = df.iloc[index_train]
df_train = df_train.reset_index()

In [5]:
df_train.head()
from sklearn.preprocessing import LabelEncoder
'''
Encoding categorical variables here
'''


from sklearn.preprocessing import LabelEncoder
def encode(highGTOData):
    lb_make = LabelEncoder()
    highGTOData['sdfc_Tier'] = lb_make.fit_transform(highGTOData['sdfc_Tier'])
    for i in range(len(highGTOData['GTO_2019'])):
        if(highGTOData['poc_image'][i]==0):
            highGTOData['poc_image'][i] = "Mainstream"
    highGTOData['poc_image'] = lb_make.fit_transform(highGTOData['poc_image'])
    highGTOData['segment'] = lb_make.fit_transform(highGTOData['segment'])
    highGTOData['sub_segment'] = lb_make.fit_transform(highGTOData['sub_segment'])
    highGTOData['Product Set'] = lb_make.fit_transform(highGTOData['Product Set'])
    highGTOData['Brand'] = lb_make.fit_transform(highGTOData['Brand'])
    highGTOData['Sub-Brand'] = lb_make.fit_transform(highGTOData['Sub-Brand'])
    highGTOData['Pack_Type'] = lb_make.fit_transform(highGTOData['Pack_Type'])
    highGTOData['Returnalility'] = lb_make.fit_transform(highGTOData['Returnalility'])
    highGTOData['province'] = lb_make.fit_transform(highGTOData['province'])
    highGTOData['GTO_growth'] = highGTOData['Expected_GTO'] - highGTOData['GTO_2019']
    return highGTOData


df_test = encode(df_test)
df_train = encode(df_train)

C:\Users\NISARG\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:



lowGTOData_train = df_train[df_train.GTO_2019<10000]
lowGTOData_test = df_test[df_test.GTO_2019<10000]

midGTOData_train = df_train[(df_train.GTO_2019>10000)&(df_train.GTO_2019<50000)]
midGTOData_test = df_test[(df_test.GTO_2019>10000)&(df_test.GTO_2019<50000)]

highGTOData_train = df_train[df_train.GTO_2019>50000]
highGTOData_test = df_test[df_test.GTO_2019>50000]

In [7]:
'''
Defining the class of machine learning models that we are going to try here
'''


class Models(object):
    
    global seed 
    seed = 34234
    
    # Initialization 
    def __init__(self, x_train, x_validation, y_train, y_validation):
        # changing input as dataframe to list
        self.x_train = [x_train.iloc[i].tolist() for i in range(len(x_train))]
        self.x_validation = [x_validation.iloc[i].tolist() for i in range(len(x_validation))]
        self.y_train = y_train.tolist()
        self.y_validation = y_validation.tolist()
    
    
    @staticmethod
    def print_info(cross_val_scores, mse):
        print("Cross Validation Scores: ", cross_val_scores)
        print("Mean Squared Error: ", mse)
        
        
    # Linear Regression 
    def linear_regression(self, x_train, x_validation,  y_train, y_validation):
        reg = linear_model.LinearRegression()
        # X = np.array(X).reshape([-1, 1])
        reg.fit(self.x_train, self.y_train)
        y_pred_list = reg.predict(self.x_validation)
        mse = math.sqrt(mean_squared_error(self.y_validation, y_pred_list))
        kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=seed)
        cross_val_scores = cross_val_score(reg, self.x_train, self.y_train, cv=kfold )
        mse_train = math.sqrt(mean_squared_error(self.y_train,reg.predict(self.x_train)))
        print("TRAINING ERROR = " , mse_train)
        print("\nLinear Regression Model")
        self.print_info(cross_val_scores, mse)
        return cross_val_scores, mse
        
    # Random Forest Regression model 
    def random_forest(self, x_train, x_validation,  y_train, y_validation):
        rfr = RandomForestRegressor(n_estimators=8, max_depth=8, random_state=12, verbose=0)
        # X = np.array(X).reshape([-1, 1])
        rfr.fit(self.x_train, self.y_train)
        y_pred_list = rfr.predict(self.x_validation)
        mse = math.sqrt(mean_squared_error(self.y_validation, y_pred_list))
        kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=seed)
        cross_val_scores = cross_val_score(rfr, self.x_train, self.y_train, cv=kfold )
        mse_train = math.sqrt(mean_squared_error(self.y_train,rfr.predict(self.x_train)))
        print("\nRandom Forest Regressor")
        print("TRAINING ERROR = " , mse_train)
        self.print_info(cross_val_scores, mse)
        return cross_val_scores, mse
            
    # Lasso method 
    def lasso(self, x_train, x_validation,  y_train, y_validation):
        reg = linear_model.Lasso(alpha = 0.1)
        # X = np.array(X).reshape([-1, 1])
        reg.fit(self.x_train, self.y_train)
        y_pred_list = reg.predict(self.x_validation)
        mse = math.sqrt(mean_squared_error(self.y_validation, y_pred_list))
        kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=seed)
        cross_val_scores = cross_val_score(reg, self.x_train, self.y_train, cv=kfold )
        mse_train = math.sqrt(mean_squared_error(self.y_train,reg.predict(self.x_train)))
        print("\nLasso Regression Model")
        print("TRAINING ERROR = " , mse_train)
        self.print_info(cross_val_scores, mse)
        return cross_val_scores, mse
    
    # Ridge method 
    def ridge(self, x_train, x_validation,  y_train, y_validation):
        reg = linear_model.Ridge(alpha = 0.1)
        # X = np.array(X).reshape([-1, 1])
        reg.fit(self.x_train, self.y_train)
        y_pred_list = reg.predict(self.x_validation)
        mse = math.sqrt(mean_squared_error(self.y_validation, y_pred_list))
        kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=seed)
        cross_val_scores = cross_val_score(reg, self.x_train, self.y_train, cv=kfold )
        mse_train = math.sqrt(mean_squared_error(self.y_train,reg.predict(self.x_train)))
        print("\nRidge Regression Model")
        print("TRAINING ERROR = " , mse_train)
        self.print_info(cross_val_scores, mse)
        return cross_val_scores, mse
    
    
     # CART method 
    def CART(self, x_train, x_validation,  y_train, y_validation):
        reg =  tree.DecisionTreeRegressor()
        # X = np.array(X).reshape([-1, 1])
        reg.fit(self.x_train, self.y_train)
        y_pred_list = reg.predict(self.x_validation)
        mse = math.sqrt(mean_squared_error(self.y_validation, y_pred_list))
        kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=seed)
        cross_val_scores = cross_val_score(reg, self.x_train, self.y_train, cv=kfold )
        mse_train = math.sqrt(mean_squared_error(self.y_train,reg.predict(self.x_train)))
        print("\nCART Regression Model")
        print("TRAINING ERROR = " , mse_train)
        self.print_info(cross_val_scores, mse)
        return cross_val_scores, mse
    
    # Gradient Boosing Regressor
    def GBR(self, x_train, x_validation,  y_train, y_validation):
        gbr = GradientBoostingRegressor(n_estimators=175, learning_rate=0.08, max_depth=3, random_state=1232, loss='ls')
        gbr.fit(self.x_train, self.y_train)
        kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=seed)
        cross_val_scores = cross_val_score(gbr, self.x_train, self.y_train, cv=kfold )
        mse_train = math.sqrt(mean_squared_error(self.y_train,gbr.predict(self.x_train)))
        mse = math.sqrt(mean_squared_error(self.y_validation, gbr.predict(self.x_validation)))
        print("mse = ",mse)
        print('\nGradient Boosting Regressor')
        print("TRAINING ERROR = " , mse_train)
        
        self.print_info(cross_val_scores, mse)
        return cross_val_scores, mse

In [8]:
lowGTOData_train = lowGTOData_train.reset_index()
lowGTOData_test = lowGTOData_test.reset_index()
target_train = lowGTOData_train['OnInvoice Discount(LCU)']
target_test = lowGTOData_test['OnInvoice Discount(LCU)']
colsToKeep = ['Volume_2019' , 'Volume_2018'  , 'Expected_GTO'  , 'Expected_product_volume', 'profitability_indicator' , 'upper_limit'  ,'sdfc_Tier'  , 'loyalty_index' , 'Returnalility', 'market_cap' ]
features_train = lowGTOData_train[colsToKeep]
features_test = lowGTOData_test[colsToKeep]

In [9]:
methods = [x for x, y in Models.__dict__.items() if type(y) == FunctionType]
methods.remove('__init__')
for model in methods:
    reg = Models(features_train, features_test, target_train, target_test)
    cross_val_scores, mse = getattr(reg, model)(features_train, features_test, target_train, target_test)
    print("MEAN CROSS VAL SCORES=" , np.mean(cross_val_scores))

TRAINING ERROR =  209.17050216238943

Linear Regression Model
Cross Validation Scores:  [ 2.75836720e-01  1.97793669e-02  8.83971554e-02  2.56159582e-01
  2.74671339e-01 -1.52580988e+10  2.67604878e-01 -1.38925911e+01
  2.43688677e-01  2.56289702e-01]
Mean Squared Error:  375.71502450074723
MEAN CROSS VAL SCORES= -1525809883.9408245

Random Forest Regressor
TRAINING ERROR =  137.3550907424985
Cross Validation Scores:  [0.52995426 0.56266093 0.56374022 0.47013767 0.50670935 0.46617072
 0.55730387 0.54710638 0.55976663 0.53439179]
Mean Squared Error:  349.9925860547361
MEAN CROSS VAL SCORES= 0.5297941807492081

Lasso Regression Model
TRAINING ERROR =  209.20852123400243
Cross Validation Scores:  [ 2.75757192e-01  1.89057285e-02  8.92729989e-02  2.56919834e-01
  2.75304480e-01 -1.53942126e+10  2.66349803e-01 -1.38888770e+01
  2.44542818e-01  2.56388663e-01]
Mean Squared Error:  375.7369180579738
MEAN CROSS VAL SCORES= -1539421261.0304525


C:\Users\NISARG\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.56443e-19): result may not be accurate.
  overwrite_a=True).T
C:\Users\NISARG\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.47388e-19): result may not be accurate.
  overwrite_a=True).T
C:\Users\NISARG\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.4355e-19): result may not be accurate.
  overwrite_a=True).T
C:\Users\NISARG\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.36404e-19): result may not be accurate.
  overwrite_a=True).T
C:\Users\NISARG\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.342


Ridge Regression Model
TRAINING ERROR =  209.17050411953647
Cross Validation Scores:  [ 2.75837755e-01  1.97715083e-02  8.84059875e-02  2.56170248e-01
  2.74678590e-01 -1.52591021e+10  2.67597753e-01 -1.38925969e+01
  2.43698312e-01  2.56292851e-01]
Mean Squared Error:  375.71475266653783
MEAN CROSS VAL SCORES= -1525910211.5622506

CART Regression Model
TRAINING ERROR =  7.365139052817077e-16
Cross Validation Scores:  [ 0.02410195  0.28601802  0.13867957  0.12418402  0.07608494 -0.08561846
  0.08596407  0.26440041  0.0740126   0.1164679 ]
Mean Squared Error:  387.59867136369564
MEAN CROSS VAL SCORES= 0.11042950194673126
mse =  340.7099992320587

Gradient Boosting Regressor
TRAINING ERROR =  131.00147197111852
Cross Validation Scores:  [0.52847839 0.53471941 0.57589338 0.49151801 0.56957272 0.51063966
 0.56089954 0.51839653 0.59377564 0.55715815]
Mean Squared Error:  340.7099992320587
MEAN CROSS VAL SCORES= 0.5441051422106856


# Comments for lowGTOData :
1) Since our training data is correct data (data where discounts are correctly given ) , we would like our Machine Learning model to give minimum training error.

2) For selecting the best model , we use the criteria : Less Training Error , high R^2 values

3) CART Model gives least training error but we can see that cross validation scores are not good , this means that the model is overfitting

4) Gradient Boosting Regressor performs the best in both Training Error and Cross Validation Scores Parameters

In [10]:
'''
Keeping the features as seen from EXPLORATORY DATA ANALYSIS
'''

midGTOData_train = midGTOData_train.reset_index()
midGTOData_test = midGTOData_test.reset_index()
target_train = midGTOData_train['Discount_Total']
target_test = midGTOData_test['Discount_Total']
colsToKeep = ['Volume_2019' , 'Volume_2018' ,'Volume_2019 Product' ,'Expected_GTO','Expected_product_volume' , 'profitability_indicator' , 'upper_limit'  ,'sdfc_Tier'  , 'loyalty_index' , 'Returnalility',  'inventory_lingering_factor', 'market_cap',
       'order_size']
features_train = midGTOData_train[colsToKeep]
features_test = midGTOData_test[colsToKeep]

In [11]:
methods = [x for x, y in Models.__dict__.items() if type(y) == FunctionType]
methods.remove('__init__')
for model in methods:
    reg = Models(features_train, features_test, target_train, target_test)
    cross_val_scores, mse = getattr(reg, model)(features_train, features_test, target_train, target_test)
    print("MEAN CROSS VAL SCORES=" , np.mean(cross_val_scores))

TRAINING ERROR =  2157.655855188812

Linear Regression Model
Cross Validation Scores:  [0.63800041 0.23246567 0.38247191 0.31567495 0.41035225 0.52893829
 0.47113808 0.39799634 0.50792226 0.47252448]
Mean Squared Error:  6583.829538001789
MEAN CROSS VAL SCORES= 0.43574846519336735

Random Forest Regressor
TRAINING ERROR =  1424.0698486921524
Cross Validation Scores:  [0.57852706 0.31551433 0.47641717 0.35457732 0.33568022 0.65900386
 0.62967686 0.38368241 0.52126377 0.49137169]
Mean Squared Error:  6414.869674902147
MEAN CROSS VAL SCORES= 0.4745714696634106

Lasso Regression Model
TRAINING ERROR =  2157.662605321827
Cross Validation Scores:  [0.63803764 0.23291445 0.38248092 0.31609428 0.41054297 0.52883927
 0.47129861 0.398313   0.50578807 0.47294024]
Mean Squared Error:  6583.274571440101
MEAN CROSS VAL SCORES= 0.4357249459524747

Ridge Regression Model
TRAINING ERROR =  2158.0184294360824
Cross Validation Scores:  [0.63808368 0.23631229 0.38197558 0.31789365 0.4114996  0.52757936
 0

# Comments for mid GTO Data
1) CART REGRESSION MODEL OVERFITS , EVEN THOUGH WE HAVE zero Training error but cross validation scores are very less

2) Gradient boosting Regressor has less training error and the highest k fold cross validation score

3) GBR model fits best for mid GTO Data

In [12]:
'''
Keeping the features as seen from EXPLORATORY DATA ANALYSIS
'''


highGTOData_train = highGTOData_train.reset_index()
highGTOData_test = highGTOData_test.reset_index()
target_train = highGTOData_train['Discount_Total']
target_test = highGTOData_test['Discount_Total']
colsToKeep = ['Volume_2019' , 'Volume_2018' ,'Volume_2019 Product' ,'Expected_GTO','Expected_product_volume' , 'profitability_indicator' , 'upper_limit'  ,  'inventory_lingering_factor',
       'order_size']
features_train = highGTOData_train[colsToKeep]
features_test = highGTOData_test[colsToKeep]

In [13]:
methods = [x for x, y in Models.__dict__.items() if type(y) == FunctionType]
methods.remove('__init__')
for model in methods:
    reg = Models(features_train, features_test, target_train, target_test)
    cross_val_scores, mse = getattr(reg, model)(features_train, features_test, target_train, target_test)
    print("MEAN CROSS VAL SCORES=" , np.mean(cross_val_scores))

TRAINING ERROR =  11592.764707271046

Linear Regression Model
Cross Validation Scores:  [-1.06087019 -0.36632146  0.97420802  0.72035285  0.62682464 -0.06819141
  0.49322678 -2.2610839   0.87362034  0.91377639]
Mean Squared Error:  124995.12651733206
MEAN CROSS VAL SCORES= 0.0845542055601822

Random Forest Regressor
TRAINING ERROR =  5966.083708326815
Cross Validation Scores:  [-1.74860924  0.5434003   0.91425773  0.85503962  0.62815798 -0.09866692
  0.46652807 -4.63388039  0.8523974   0.92125103]
Mean Squared Error:  171986.4990619022
MEAN CROSS VAL SCORES= -0.1300124437627878


C:\Users\NISARG\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8735745248.855309, tolerance: 17511858.16904576
  positive)
C:\Users\NISARG\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5648836278.466648, tolerance: 16989957.83912539
  positive)
C:\Users\NISARG\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6541686295.815524, tolerance: 17093948.063802466
  positive)
C:\Users\NISARG\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWa


Lasso Regression Model
TRAINING ERROR =  11592.8733428284
Cross Validation Scores:  [-1.07080799 -0.36169851  0.97953099  0.72041248  0.62548444 -0.07059253
  0.48980501 -2.07415161  0.87716266  0.91381811]
Mean Squared Error:  124878.77966440414
MEAN CROSS VAL SCORES= 0.1028963054053718

Ridge Regression Model
TRAINING ERROR =  11592.81823974513
Cross Validation Scores:  [-1.05852978 -0.36697204  0.97403141  0.72024496  0.62874838 -0.06852993
  0.49177446 -2.12594403  0.87364746  0.91384709]
Mean Squared Error:  125016.88051406432
MEAN CROSS VAL SCORES= 0.09823179709549928

CART Regression Model
TRAINING ERROR =  0.0
Cross Validation Scores:  [ -1.59291365   0.3146687    0.95990044   0.74736021   0.49807266
  -0.89021095   0.15780425 -29.12889898   0.83467154   0.89431004]
Mean Squared Error:  169473.75763667212
MEAN CROSS VAL SCORES= -2.7205235737808118
mse =  174374.6010934097

Gradient Boosting Regressor
TRAINING ERROR =  1573.8869325240246
Cross Validation Scores:  [-1.86965283  

# Comments for High GTO Data
1) Linear Models are seen to have better cross validation scores as compared to GBR even though they have higher training error

2) Combining results of GBR & Lasso could be an ideal choice for High GTO Data

# What if OnInvoice Discount Predicted is less than predicted Total Discount  (given Total Discount is +ve) ?

Ans : Since we know that Total Discount can be spreaded into 2 components & hence split the remaining into Off Invoice Discount

# What if OnInvoice Discount Predicted is more than predicted Total Discount (given Total Discount is +ve) ?
 
Ans : We would have to adjust the OnInvoice Discount offered --- Make OnInvoice Discount = Predicted Total Discount & OffInvoice Discount = 0
